## FIFA World ranking analysis based on country GDP Data preprocessing

In this Notebook we will do Data Preprocessing, and later that data will be used by our Dash application.

First let's import our libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
from io import StringIO
import re
import numpy as np

### Getting the Data

One of our datasets is downloaded from kaggle.com, which we will read into oandas DataFrame below, the other one we will scrape from the web.

The one downloaded is the FIFA Ranking of the countries from 1993, let's see what it looks like

In [2]:
fifa = pd.read_csv("fifa_ranking.csv")

In [3]:
fifa.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


Now let's scrape the other dataset from wikipedia. For that we will use Beautiful Soup library.

In [4]:
def make_soup(url):
    thepage=urllib.request.urlopen(url)
    soupdata=BeautifulSoup(thepage, "html.parser")
    return soupdata

In [5]:
soup = make_soup("https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(nominal)")

In [6]:
def create_csv(table):
    countries_data_saved = ""
    for record in table.findAll('tr'):
        countries_data = ""
        for data in record.findAll('th'):
            countries_data = countries_data + "_" + data.text
        for data in record.findAll('td'):
            countries_data = countries_data + "_" + data.text
        countries_data_saved = countries_data_saved + countries_data[1:]

    countries_data_saved = countries_data_saved.replace(",", "")
    countries_data_saved = countries_data_saved.replace("\xa0", "")
    countries_data_saved = countries_data_saved.replace("_", ",")
    
    return countries_data_saved
    

tables = soup.find_all('table', class_="wikitable")
csv1 = create_csv(tables[1])
countries_1 = pd.read_csv(StringIO(csv1))

csv2 = create_csv(tables[3])
countries_2 = pd.read_csv(StringIO(csv2))

csv3 = create_csv(tables[5])
countries_3 = pd.read_csv(StringIO(csv3))

csv4 = create_csv(tables[7])
countries_4 = pd.read_csv(StringIO(csv4))

In [7]:
countries = pd.merge(countries_1.drop_duplicates(), countries_2.drop_duplicates(), on='Country (or dependent territory)', validate="one_to_one", how="inner")
countries = pd.merge(countries, countries_3.drop_duplicates(), on='Country (or dependent territory)', validate="one_to_one", how="inner")
countries = pd.merge(countries, countries_4.drop_duplicates(subset='Country (or dependent territory)', keep="last"), on='Country (or dependent territory)', validate="one_to_one", how="inner")

countries = pd.melt(countries, id_vars=['Country (or dependent territory)'], var_name='year', value_name='gdp')
countries.columns = ["country", "year", "gdp"]

### Data Cleaning

Now let's clean the datasets. At first let's remove month and day from the date of FIFA datase, and do type conversions.

In [8]:
fifa.rank_date = fifa.rank_date.apply(lambda x: re.sub(r'-[0-9]{2}-[0-9]{2}', "", x))
fifa.rank_date = fifa.rank_date.astype(int)
countries.year = countries.year.astype(int)

Now, since FIFA dataset contains Rankings for years 1993-2018, we need to remove valued from Countried dataset that are out from that scope

In [9]:
countries = countries[(countries.year >= 1993) & (countries.year <= 2018)]

Since we have several rank values in FIFA Dataset for the same country and same year, we will keep only the mean of the rank for several values.

In [10]:
fifa = fifa.groupby(['rank_date', "country_full"], as_index=False)['rank'].mean()
fifa.columns.values[1] = 'country'
fifa.columns.values[0] = 'year'

There are Countries that are present only in one of the datasets, let's remove them

In [11]:
combined = fifa.merge(countries, on=["country", "year"], how='left', indicator=True, validate='one_to_one')
fifa = fifa[combined['_merge'] == 'both']

In [12]:
combined = countries.merge(fifa, on=["country", "year"], how='left', indicator=True, validate='one_to_one')
countries.reset_index(inplace=True, col_fill=False)
countries = countries[combined['_merge'] == 'both']

Let's see the datasets we got

In [19]:
countries.head()

,index,country,year,gdp
1,2523,Albania,1993,1461.0
2,2524,Algeria,1993,50963.0
3,2525,Angola,1993,6095.0
4,2526,Antigua and Barbuda,1993,535.0
5,2527,Argentina,1993,256365.0


In [18]:
fifa.head()

,year,country,rank
0,1993,Albania,91.8
1,1993,Algeria,32.6
2,1993,Angola,97.4
3,1993,Antigua and Barbuda,125.8
4,1993,Argentina,8.6


Now let's Save the datsets into CSVs

In [15]:
fifa.to_csv("fifa_ranking_preprocessed.csv")
countries.to_csv("gdp_preprocessed.csv")

This data will now be used in our Dash applicaion